# Entraînement d'un modèle LSTM avec BERT pour la classification de texte

*Membres du groupe* :
- ADAM EZ-ZAHIR
- ACHRAF JEMALI
- ANGE-MARIE GOUNADON
- SOUROU ALFRED SOUDJI


Dans ce projet, nous nous pencherons sur le développement d'un modèle de classification de texte en utilisant une approche plus avancée. Notre choix se porte sur l'intégration du modèle BERT (Bidirectional Encoder Representations from Transformers) dans une architecture LSTM (Long Short-Term Memory).


**Dataset**

Pour notre expérimentation, nous utiliserons le jeu de données AG_NEWS, une collection de titres d'articles de nouvelles accompagnés de descriptions provenant de diverses sources d'informations. Organisé en quatre catégories principales (Monde, Sports, Business et Science/Technologie), ce jeu de données offre un cadre propice à la classification de texte, permettant à notre modèle de prédire automatiquement la catégorie d'une nouvelle donnée.

**Préparation de l'environnement**

Nous débuterons par la configuration de notre environnement de développement en important les bibliothèques nécessaires, notamment PyTorch pour la modélisation, Transformers pour l'intégration de BERT, et d'autres modules essentiels.

**Chargement des données**

Le jeu de données AG_NEWS sera chargé en utilisant les utilitaires de torchtext, simplifiant le processus de téléchargement et de prétraitement des données textuelles.

**Prétraitement du texte**

Nous aborderons les étapes de tokenisation et de construction de vocabulaire, tout en adaptant ces processus à l'intégration de BERT pour assurer une représentation adéquate des données.

**Construction du modèle LSTM avec BERT**

Définir la classe du modèle LSTMClassifier sera notre prochaine étape. Cette classe incorporera l'architecture LSTM, tirant parti des fonctionnalités de BERT pour une compréhension contextuelle plus profonde du texte.

**Entraînement du modèle**

Nous procéderons à l'entraînement du modèle en utilisant la technique de rétropropagation à travers le temps (BPTT). Nous ajusterons la fonction de perte et les paramètres d'optimisation pour garantir des performances optimales.

**Évaluation du modèle**

Enfin, nous évaluerons les performances du modèle en calculant la précision sur un ensemble de test. Cela nous permettra d'apprécier la capacité prédictive du modèle dans des conditions similaires à celles d'une application réelle.

Notre approche combinant LSTM et BERT vise à exploiter les avantages de chacun de ces modèles pour obtenir des résultats plus performants dans la classification de texte.



In [7]:
!pip install portalocker>=2.0.0

In [8]:
# Importation des modules nécessaires
from torchtext.datasets import AG_NEWS
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch import nn
from transformers import BertModel, BertTokenizer
import torch.optim as optim

# Crée un itérateur pour le jeu de données AG News avec l'ensemble d'entraînement
train_iter = iter(AG_NEWS(split='train'))

# Récupère le premier exemple du jeu de données
first_example = next(train_iter)

# Affiche le premier exemple
print(first_example)

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


Le code suivant définit une classe `BertLSTMClassifier` qui combine un modèle BERT pré-entraîné avec une couche LSTM pour la classification de texte. L'objectif est de capturer des représentations riches du texte avec BERT et de les traiter avec une couche LSTM pour effectuer la classification. Les paramètres de BERT sont figés pour conserver les poids pré-entraînés. La méthode `forward` spécifie le flux de données à travers le modèle lors de la propagation avant. Le modèle est conçu pour la tâche de classification de textes, en utilisant le jeu de données AG News comme exemple.

In [9]:
# Définition d'un classificateur LSTM avec BERT
class BertLSTMClassifier(nn.Module):
    def __init__(self, hidden_dim, num_classes, bert_model_name='bert-base-uncased'):
        super(BertLSTMClassifier, self).__init__()

        # Initialisation du modèle BERT pré-entraîné
        self.bert = BertModel.from_pretrained(bert_model_name)

        # Couche LSTM prenant la sortie de BERT
        self.lstm = nn.LSTM(self.bert.config.hidden_size, hidden_dim, batch_first=True)

        # Couche entièrement connectée pour la classification
        self.fc = nn.Linear(hidden_dim, num_classes)

        # Désactivation de l'apprentissage pour les paramètres de BERT
        for param in self.bert.parameters():
            param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        # Exécution de BERT sur les données d'entrée (avec la désactivation des gradients)
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Passe la sortie de BERT à travers la couche LSTM
        lstm_out, _ = self.lstm(bert_output.last_hidden_state)


        # Sélectionne la dernière sortie de la séquence LSTM
        final_out = lstm_out[:, -1, :]

        # Passe la sortie finale à travers la couche entièrement connectée pour la classification
        return self.fc(final_out)

# Utilisation de l'itérateur pour obtenir un exemple du jeu de données AG News
next(iter(train_iter))



(3,
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')

le code suivant que nous avons rédigé a pour objectif l'utilisation du modèle BERT pour la classification de texte sur le jeu de données AG News. Il configure l'environnement GPU si disponible, télécharge les données d'entraînement et de test AG News, utilise le tokenizer BERT pour traiter les textes, et crée des DataLoader pour l'entraînement et le test avec un batch_size de 8. L'objectif global est de préparer les données dans un format compatible avec BERT et de créer des DataLoader pour faciliter l'entraînement d'un modèle de classification de texte avec BERT

In [10]:
# Vérifie la disponibilité de GPU et utilise CUDA s'il est disponible, sinon utilise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Télécharge les jeux de données d'entraînement et de test AG News
train_iter, test_iter = AG_NEWS()

# Initialise le tokenizer BERT avec le modèle 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour regrouper un batch de données pour l'entraînement avec BERT
def collate_batch_bert(batch):
    # Récupère les textes et les étiquettes du batch
    texts = [x[1] for x in batch]
    labels = torch.tensor([x[0]-1 for x in batch], dtype=torch.long)  # Ajuste les étiquettes de 1 à n_classes

    # Tokenisation des textes avec BERT, avec gestion du padding et troncature
    encoding = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    input_ids = encoding['input_ids']  # Identifiants d'entrée BERT
    attention_mask = encoding['attention_mask']  # Masque d'attention pour les séquences

    # Retourne les étiquettes et les données tokenisées
    return labels.to(device), input_ids.to(device), attention_mask.to(device)

# Crée des DataLoader pour l'entraînement et le test avec le batch_size de 8
train_loader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch_bert)
test_loader = DataLoader(test_iter, batch_size=8, shuffle=False, collate_fn=collate_batch_bert)


Nous avons configuré le modèle pour la classification de texte en utilisant la classe BertLSTMClassifier. Nous avons définit les dimensions pour l'embedding, la couche cachée, et le nombre de classes. Ensuite, Nous avons instancié un objet de cette classe avec les dimensions spécifiées. Enfin, Nous avons définit une fonction de perte (CrossEntropyLoss) adaptée à la classification multiclasse, et configuré un optimiseur Adam avec un taux d'apprentissage de 0.001 pour l'entraînement du modèle. L'objectif global est de préparer le modèle pour l'entraînement en spécifiant ses caractéristiques et les paramètres d'entraînement.






In [11]:
# Définition des dimensions pour l'embedding, la couche cachée, et le nombre de classes
embed_dim = 64
hidden_dim = 128
num_classes = 4

# Instanciation d'un objet de la classe BertLSTMClassifier avec les dimensions spécifiées
model = BertLSTMClassifier(hidden_dim, num_classes).to(device)

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.CrossEntropyLoss()  # Fonction de perte pour la classification multiclasse
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimiseur Adam avec un taux d'apprentissage de 0.001

Ce code implémente une boucle d'entraînement sur plusieurs époques pour un modèle de classification de texte utilisant BERT et LSTM. Il effectue la propagation avant, calcule la perte, effectue la rétropropagation, et met à jour les poids du modèle à chaque itération sur les batches d'entraînement. La précision d'entraînement et de test est surveillée et affichée après chaque époque. L'objectif global est d'entraîner le modèle et d'évaluer ses performances sur les données de test.

In [6]:
from tqdm import tqdm

# Boucle d'entraînement sur 5 époques
for epoch in range(5):
    model.train()  # Met le modèle en mode d'entraînement
    total_loss = 0
    num_batches = 0
    correct_preds = 0  # Nombre de prédictions correctes
    total_preds = 0  # Nombre total de prédictions

    # Barre de progression pour l'époque en cours
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}')

    model.train()  # Met à nouveau le modèle en mode d'entraînement (au cas où)

    # Boucle sur les batches d'entraînement
    for labels, input_ids, attention_mask in progress_bar:
        num_batches += 1
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()  # Réinitialise les gradients
        output = model(input_ids, attention_mask)  # Propagation avant

        loss = criterion(output, labels)  # Calcul de la perte
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids du modèle

        total_loss += loss.item()  # Accumulation de la perte

        _, predicted_labels = torch.max(output, 1)  # Sélection des prédictions

        correct_preds += (predicted_labels == labels).sum().item()  # Calcul des prédictions correctes
        total_preds += labels.size(0)  # Accumulation du nombre total de prédictions

        # Mise à jour de la barre de progression
        progress_bar.set_postfix({
            'Train Loss': loss.item(),
            'Train Acc': 100. * correct_preds / total_preds
        })

    # Calcul de la perte moyenne et de la précision d'entraînement
    train_loss = total_loss / num_batches
    train_accuracy = 100. * correct_preds / total_preds

    # Mise en mode évaluation pour l'évaluation sur les données de test
    model.eval()

    correct_preds = 0
    total_preds = 0

    # Boucle sur les batches de test
    for labels, input_ids, attention_mask in test_loader:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        output = model(input_ids, attention_mask)  # Propagation avant sur les données de test

        _, predicted_labels = torch.max(output, 1)  # Sélection des prédictions

        correct_preds += (predicted_labels == labels).sum().item()  # Calcul des prédictions correctes
        total_preds += labels.size(0)  # Accumulation du nombre total de prédictions

    # Calcul de la précision sur les données de test
    test_accuracy = 100 * correct_preds / total_preds

    # Affichage des résultats de l'époque
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}, Test Accuracy: {test_accuracy:.2f}%')


Epoch 1: 15000it [11:58, 20.88it/s, Train Loss=0.00434, Train Acc=90.4]


Epoch 1, Train Loss: 0.2795, Train Accuracy: 90.42, Test Accuracy: 91.26%


Epoch 2: 15000it [11:55, 20.96it/s, Train Loss=0.00264, Train Acc=92.4]


Epoch 2, Train Loss: 0.2164, Train Accuracy: 92.41, Test Accuracy: 92.03%


Epoch 3: 15000it [11:56, 20.93it/s, Train Loss=0.00329, Train Acc=93.2]


Epoch 3, Train Loss: 0.1935, Train Accuracy: 93.17, Test Accuracy: 92.25%


Epoch 4: 15000it [11:56, 20.94it/s, Train Loss=0.00134, Train Acc=93.6]


Epoch 4, Train Loss: 0.1798, Train Accuracy: 93.59, Test Accuracy: 92.51%


Epoch 5: 15000it [11:56, 20.94it/s, Train Loss=0.000518, Train Acc=94]


Epoch 5, Train Loss: 0.1702, Train Accuracy: 94.01, Test Accuracy: 92.61%
